# Tutorial: Taming Big Data With Apache Spark and Python - Hands On!
## Exercise 8 - Movie Similarity (Collaborative Filtering)

### Setup

FindSpark

This will circumvent many issues with your system finding spark

In [5]:
import findspark
findspark.init('c:/users/andy/spark')

Load Libraries

In [6]:
import sys
from pyspark import SparkConf, SparkContext
from math import sqrt

Set the file path

In [7]:
data_folder = "C:/Users/Andy/Dropbox/FactoryFloor/Repositories/Tutorial_Udemy_SparkPython/Course_Resources/ml-100k/"

Create the Spark Context

In [8]:
# configure your Spark context; master node is local machine
conf = SparkConf().setMaster("local[*]").setAppName("MovieSimilarities")

# create a spark context object
sc = SparkContext(conf = conf)

### Define Functions

In [9]:
def loadMovieNames():
    movieNames = {} # create a dict
    file_to_open = data_folder + "u.ITEM" #file path
    with open(file_to_open, encoding = 'ascii', errors = 'ignore') as f: # open file
        for line in f:
            fields = line.split('|') # break the lines
            movieNames[int(fields[0])] = fields[1] # create key-value
    return movieNames

In [42]:
def filterDuplicates(userRatings):
    ratings = userRatings[1] # the value ((movieID, rating), (movieID, rating))
    (movie1, rating1) = ratings[0] 
    (movie2, rating2) = ratings[1]
    return movie1 < movie2 # return only those entries where movieID 2 is greater than movieID 1

In [46]:
#Python 3 doesn't let you pass arond unpacked tuples,
# so we explicitly extract the ratings now.
def makePairs(userRatings):
    ratings = userRatings[1] # the value ((movieID, rating), (movieID, rating))
    (movie1, rating1) = ratings[0]
    (movie2, rating2) = ratings[1]
    return ((movie1, movie2), (rating1, rating2)) #format so its pair of movies and pair of ratings

In [50]:
def computeCosineSimilarity(ratingPairs):
    numPairs = 0
    sum_xx = sum_yy = sum_xy = 0
    for ratingX, ratingY in ratingPairs:
        sum_xx += ratingX * ratingX
        sum_yy += ratingY * ratingY
        sum_xy += ratingX * ratingY
        numPairs += 1
    
    numerator = sum_xy
    denominator = sqrt(sum_xx) * sqrt(sum_yy)
    
    score = 0
    if (denominator):
        score = (numerator / (float(denominator)))
        
    return (score, numPairs)

### The Program

**1) Create a dictionary with movieID and movieNames.**

In [27]:
print("\nLoading movie names...")
nameDict = loadMovieNames()

print("Success! Your", type(nameDict), "was created with", len(nameDict), "entries. For instance", nameDict.get(42), "is one of them.")


Loading movie names...
Success! Your <class 'dict'> was created with 1682 entries. For instance Clerks (1994) is one of them.


**2) Bring in the movie ratings data.**

In [39]:
data = sc.textFile(data_folder + "u.data")

print("Success! Your", type(data), "was created with", data.count(), "entries. \
\n\nThe first five are:\n", data.take(5))

Success! Your <class 'pyspark.rdd.RDD'> was created with 100000 entries. 

The first five are:
 ['196\t242\t3\t881250949', '186\t302\t3\t891717742', '22\t377\t1\t878887116', '244\t51\t2\t880606923', '166\t346\t1\t886397596']


**3) Key/value stores of dicts: user ID => movie ID, rating.**

In [40]:
ratings = data.map(lambda l: l.split()).map(lambda l: (int(l[0]), (int(l[1]), float(l[2]))))

print("Success! Your", type(ratings), "was created with", ratings.count(), "entries. \
\n\nThe first five are:\n", ratings.take(5))

Success! Your <class 'pyspark.rdd.PipelinedRDD'> was created with 100000 entries. 

The first five are:
 [(196, (242, 3.0)), (186, (302, 3.0)), (22, (377, 1.0)), (244, (51, 2.0)), (166, (346, 1.0))]


**4) Emit every movie rated together by the same user. Self-join to find every combination.**

In [41]:
joinedRatings = ratings.join(ratings)

print("Success! Your", type(joinedRatings), "was created with", joinedRatings.count(), "entries. \
\n\nThe first five are:\n", joinedRatings.take(5))

Success! Your <class 'pyspark.rdd.PipelinedRDD'> was created with 20200812 entries. 

The first five are:
 [(196, ((242, 3.0), (242, 3.0))), (196, ((242, 3.0), (393, 4.0))), (196, ((242, 3.0), (381, 4.0))), (196, ((242, 3.0), (251, 3.0))), (196, ((242, 3.0), (655, 5.0)))]


**5) Filter out duplicate pairs**

In [45]:
uniqueJoinedRatings = joinedRatings.filter(filterDuplicates)

print("Success! Your", type(uniqueJoinedRatings), "was created with", uniqueJoinedRatings.count(), "entries. \
\n\nThe filter removed", joinedRatings.count() - uniqueJoinedRatings.count(), "entries.\
\n\nThe first five are:\n", uniqueJoinedRatings.take(5))

Success! Your <class 'pyspark.rdd.PipelinedRDD'> was created with 10050406 entries. 

The filter removed 10150406 entries.

The first five are:
 [(196, ((242, 3.0), (393, 4.0))), (196, ((242, 3.0), (381, 4.0))), (196, ((242, 3.0), (251, 3.0))), (196, ((242, 3.0), (655, 5.0))), (196, ((242, 3.0), (306, 4.0)))]


**5) Now key by (movie1, movie2) pairs**

In [48]:
moviePairs = uniqueJoinedRatings.map(makePairs)

print("Success! Your", type(moviePairs), "was created with", moviePairs.count(), "entries. \
\n\nThe first five are:\n", moviePairs.take(5))

Success! Your <class 'pyspark.rdd.PipelinedRDD'> was created with 10050406 entries. 

The first five are:
 [((242, 393), (3.0, 4.0)), ((242, 381), (3.0, 4.0)), ((242, 251), (3.0, 3.0)), ((242, 655), (3.0, 5.0)), ((242, 306), (3.0, 4.0))]


**6) Group by key.**

In [49]:
moviePairRatings = moviePairs.groupByKey()

print("Success! Your", type(moviePairRatings), "was created with", moviePairRatings.count(), "entries. \
\n\nThe first five are:\n", moviePairRatings.take(5))

Success! Your <class 'pyspark.rdd.PipelinedRDD'> was created with 983206 entries. 

The first five are:
 [((242, 580), <pyspark.resultiterable.ResultIterable object at 0x000002076C3CD748>), ((242, 692), <pyspark.resultiterable.ResultIterable object at 0x000002076C421128>), ((242, 428), <pyspark.resultiterable.ResultIterable object at 0x000002076C421CF8>), ((242, 340), <pyspark.resultiterable.ResultIterable object at 0x000002076C421400>), ((393, 1241), <pyspark.resultiterable.ResultIterable object at 0x000002076C421710>)]


**7) Compute similarities.**

*Save the results if desired
moviePairSimilarities.sortByKey()
moviePairSimilarities.saveAsTextFile("movie-sims")*

In [51]:
moviePairSimilarities = moviePairRatings.mapValues(computeCosineSimilarity).cache()

print("Success! Your", type(moviePairSimilarities), "was created with", moviePairSimilarities.count(), "entries. \
\n\nThe first five are:\n", moviePairSimilarities.take(5))

Success! Your <class 'pyspark.rdd.PipelinedRDD'> was created with 983206 entries. 

The first five are:
 [((242, 580), (0.9443699330874624, 6)), ((242, 692), (0.9203762039948743, 18)), ((242, 428), (0.9419097988977888, 15)), ((242, 340), (0.9455404837184603, 32)), ((393, 1241), (1.0, 1))]


Extract similarities for the movie we care about that are "good".

In [53]:
len(sys.argv)

3

In [55]:
print(sys.argv)

['C:\\Users\\Andy\\Anaconda3\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\Andy\\AppData\\Roaming\\jupyter\\runtime\\kernel-ec43cdaa-d7ba-4395-a03b-e4277274e23f.json']


In [52]:
if (len(sys.argv) > 1):
    
    scoreThreshold = 0.97
    coOccurenceThreshold = 50
    
    movieID = int(sys.argv[1])
    
    # Filter for movies with this sim that are "good" as defined by 
    # our quality thresholds above
    filteredResults = moviePairSimilarities.filter(lambda pairSim: \
                                                  (pairSim[0][0] == movieID or pairSim[0][1] == movieID) \
                                                  and pairSim[1][0] > scoreThreshold and pairsim[1][1] > coOccurenceThreshold)
    
    # Sort by quality score.
    results = filteredResults.map(lambda pairSim: (pairSim[1], pairSim[0])).sortByKey(ascending= False).take(10)
    
    print("Top 10 similar movies for " + nameDict[movieID])
    for result in results:
        (sim, pair) = result
        # Display the similarity result that isn't the movie we're looking at
        similarMovieID = pair[0]
        if (similarMovieID == movieID):
            similarMovieID == pair[1]
        print(nameDict[similarMovieID] + "\t score: " + str(sim[0]) + "\t strength: " + str(sim[1]))

ValueError: invalid literal for int() with base 10: '-f'